In [ ]:
import pandas as pd  
pd.options.plotting.backend = 'plotly'
pd.set_option('display.max_columns', None)

import numpy as np            

from scipy.stats import poisson        
from scipy.optimize import minimize    
from scipy.special import factorial

from numba import jit 

from sklearn.preprocessing import StandardScaler

import plotly
import plotly.express as px

**Loglike Functions**

In [ ]:
@jit()
def linear_loglike(price, count, alfa, beta):
    '''
    Linear Log-Likelihood from demand linear function:
    
    demand = alfa*price + beta
    
    args:
    price float : product price
    count int : product quantity sold in that price
    alfa float : first fit parameter, slope of linear model
    beta float : second fit parameter, intercept of linear model
    '''
    
    demand = alfa*price + beta
    
    return -demand + count*np.log(demand) #- np.log(factorial(count))  
    

In [ ]:
@jit()
def exponential_loglike(price, count, alfa, beta):
    '''
    Linear Log-Likelihood from demand exponential function:
    
    demand = np.exp(alfa*price + beta)
    
    args:
    price float : product price
    count int : product quantity sold in that price
    alfa float : first fit parameter, price parameter
    beta float : second fit parameter, intercept parameter
    '''
    
    demand = np.exp(alfa*price + beta)
    
    return -demand + count*np.log(demand) # - np.log(factorial(count))  

In [ ]:
@jit()
def elasticity_loglike(price, count, alfa, beta):
    '''
    Linear Log-Likelihood from demand elasticity model:
    
    demand = alfa*price**beta
    
    args:
    price float : product price
    count int : product quantity sold in that price
    alfa float : first fit parameter, price parameter
    beta float : second fit parameter, elasticity parameter
    '''
    
    #demand = alfa*price**beta
    
    
    demand = np.exp(np.log(alfa)+beta*np.log(price))
    
    
    return -demand + count*np.log(demand) # - np.log(factorial(count))  
    
    
    

**Fit Model**

In [ ]:
def fit(datos, loglike, p1, p2,
        opciones={'disp':True,'maxiter':100},  # Funcion para obtener alfas, betas y gamma, stats y maximas iteraciones
        metodo='trust-constr',    # metodo Nelder-Mead, Powell, CG, BFGS, Newton-CG, L-BFGS-B, TNC, COBYLA, SLSQP, trust-constr, dogleg, trust-ncg, trust-exact, trust-krylov (tambien custom)
        ):  

    
    # inicializacion aleatoria de los parametros
    val_inicial=np.concatenate((np.array([p1]),   # alfa
                                np.array([p2])    # beta
                                ))                
  

    def menos_loglike(params):  # Funcion menos log-verosimilitud, funcion a minimizar

        log_like = [loglike(n.price,      # price from data
                            n.count_,     # count from data
                            params[0],    # alfa parameter
                            params[1])    # beta parameter
                    for n in datos.itertuples()] # seleccion en filas
        
        return -sum(log_like) 
        
    
    salida=minimize(menos_loglike, 
                    val_inicial, 
                    options=opciones, 
                    #constraints=restricciones, 
                    method=metodo) # minimizacion

    #return salida       # salida de la minimizacion
    return salida.x   # alfa, beta 
    

**DATA**

In [ ]:
%%time

df = pd.read_parquet('../data/bbva.parquet')

df.head()

In [ ]:
data = df[['cutoff_date', 'extracash_turnover_amount', 'extracash_turnover_rate']].dropna()

In [ ]:
data.extracash_turnover_rate=data.extracash_turnover_rate.round()

In [ ]:
data.extracash_turnover_amount.plot.box()

In [ ]:
data.head()

In [ ]:
data = data[data.extracash_turnover_amount<10_105]

In [ ]:
data=data.groupby(['cutoff_date', 'extracash_turnover_rate']).count()

In [ ]:
dates=sorted(list(set([e[0] for e in data.index])))

In [ ]:
prices=list(set([e[1] for e in data.index]))

In [ ]:
raw_data = []


for d in dates:
    
    for p in prices:
        
        try:
            item = {'date': d, 'price': p, 'count': data.loc[d].loc[p][0]}
            raw_data.append(item)
            
        except:
            item = {'date': d, 'price': p, 'count': 0}
            raw_data.append(item)
            


In [ ]:
data=pd.DataFrame(raw_data)

In [ ]:
#data=data.reset_index()
#data.columns = ['date', 'price', 'count']

In [ ]:
data.head()

In [ ]:
data.price.plot.box()

In [ ]:
data['count'].plot.box()

In [ ]:
#data = data[(data.price>0) & (data.price<100)]

#data = data[data['count']<351]

data['count'] = data['count'].apply(lambda x: x if x<314 else 39)

In [ ]:
data['count'].plot.box()

In [ ]:
def normalizer(x):
    
    x_mean = x.mean()
    
    x_std = x.std()
    
    
    return (x-x_mean)/x_std
    

In [ ]:
#data.price = normalizer(data.price)

data.price = data.price/10

data.head()

In [ ]:
data.shape

In [ ]:
data['count'].plot.box()

In [ ]:
data2 = data.copy()

data2.price = data.price + 5

data = pd.concat([data, data2])

In [ ]:
data = pd.read_parquet('../data/fake_data3.parquet')

data.head()

In [ ]:
#data=data[data.price>20]

In [ ]:
#data.price=data.price.round() / 10

In [ ]:
#data['count_']=0

In [ ]:
#data=data.groupby(['date', 'price']).count().reset_index()

In [ ]:
data

**Fitting**

In [ ]:
%%time

linear_params = fit(data[['price', 'count_']], 
                    linear_loglike, 
                    p1=1.,
                    p2=1.,
                    opciones={'disp':True,'maxiter':10000})

linear_params

In [ ]:
%%time

exponential_params = fit(data[['price', 'count_']], 
                         exponential_loglike, 
                         p1=-1., 
                         p2=0.,
                         opciones={'disp':True,'maxiter':10000})

exponential_params

In [ ]:
%%time

elasticity_params = fit(data[['price', 'count_']], 
                        elasticity_loglike, 
                        p1=4.,
                        p2=4.,
                        opciones={'disp':True,'maxiter':10000})

elasticity_params

**Plotting**

In [ ]:
x = np.linspace(1, 100, 100)

x

In [ ]:
linear_params

In [ ]:
linear_y = linear_params[0]*x + linear_params[1]

linear_y

In [ ]:
exp_y = np.exp(exponential_params[0]*x+exponential_params[1])

exp_y

In [ ]:
elasticity_y = elasticity_params[0]*x**elasticity_params[1]

elasticity_y

**Plots**

In [ ]:
data.describe()

In [ ]:
df_plot=pd.DataFrame()

df_plot['x']=x
df_plot['linear_y']=linear_y
df_plot['exp_y']=exp_y
df_plot['elasticity_y']=elasticity_y


px.line(df_plot, x='x', y='linear_y')

In [ ]:
px.line(df_plot, x='x', y='exp_y')

In [ ]:
df_plot['exp_profit']=df_plot.x*df_plot.exp_y*100

px.line(df_plot, x='x', y='exp_profit')

In [ ]:
df_plot

In [ ]:
px.line(df_plot, x='x', y='elasticity_y')

In [ ]:
data.describe()

In [ ]:
data